<hr><center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em>Fix pods stuck in Terminating state</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Resize-PVC&para;"><u>Pods Stuck in Terminating State</u></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview</h1>
<p><a href="#1" target="_self" rel="noopener">1)</a> <a href="#1">Get pods stuck in Terminating State</a><a href="#1" target="_self" rel="noopener"><br>2)&nbsp;</a><a href="#2">Check for finalizers</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#1" target="_self" rel="noopener">3) </a><a href="#1">Get Node Information</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#1" target="_self" rel="noopener">4)&nbsp;</a><a href="#1">Force-delete the pod</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#1" target="_self" rel="noopener">5)&nbsp;</a><a href="#1">Check Resolution</a><a href="#1" target="_self" rel="noopener"><br></a><a href="#2" target="_self" rel="noopener">6)&nbsp;</a><a href="#2">Further steps</a></p>
<p>A Pod has been deleted but remains in&nbsp;<code>Terminating</code> Status</p>
<p>This can happen for either of the reasons:</p>
<pre><code>1. Pod has a finalizer associated with it and that is not completing
2. The Pod is not responding to termination signals
</code></pre>
<p>The output of <code>kubectl get pods [PODNAME] -n [NAMESPACE]</code> will show something like this:</p>
<pre><code>NAME                     READY     STATUS             RESTARTS   AGE
nginx-7ef9efa7cd-qasd2   1/1       Terminating        0          1h
</code></pre>
<hr>

<h3 id="Convert-namespace-to-String-if-empty&para;&para;"><a id="1" target="_self" rel="nofollow"></a>Convert namespace to String if empty</h3>
<p>This custom action changes the type of namespace from None to String only if no namespace is given</p>

In [ ]:
if namespace==None:
    namespace=''

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Get List of Pods in Terminating State</h3>
<p>This action fetches a list of the pods in Terminating State. This action will consider <code>namespace</code> as&nbsp;<strong> all&nbsp;</strong>if no namespace is given.</p>
<blockquote>
<p>This action takes the following parameters (Optional):&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>terminating_pods</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} | grep Terminating | cut -d' ' -f 1\\""
    }''')

task.configure(outputName="terminatingPods")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Check for finalizer</h3>
<p>This action checks for finalizer. If it does, their failure to complete may be the root cause.</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace, terminatingPods</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>finalizerOutput</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pod -n {namespace} {terminatingPods.strip()} -o yaml | grep -A 1 finalizers\\" "
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "terminatingPods is not ''",
    "condition_result": true
    }''')
task.configure(outputName="finalizerOutput")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Remove finalizer if present</h3>
<p>This action takes input from Step 2 and removes finalizer if present</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>finalizerOutput</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl patch pod {terminatingPods.strip()}\\" + \\" -p '{\\\\\\"metadata\\\\\\":{\\\\\\"finalizers\\\\\\":null}}'\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "finalizerOutput is not ''",
    "condition_result": true
    }''')
task.configure(outputName="removeFinalizerOutput")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Get node information</h3>
<p>This action gets the nodes information to check for it's status (Step 3A)</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>nodeName</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods {terminatingPods.strip()} -n {namespace} -o yaml | grep nodeName | tr -d \\\\\\" \\\\\\" | cut -d':' -f 2\\" "
    }''')
task.configure(outputName="nodeName")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Check status of Node</h3>
<p>This action gets the status of node. It is possible that the node your pod(s) is/are running on has failed in some way. If all pods on the same node are in a&nbsp;<code>Terminating</code>&nbsp;state on a specific node, then this may be the issue.</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>nodeStatus</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace} {nodeName} | grep \\\\\\" Ready\\\\\\" | cut -d' ' -f 1 | tr -d ' '\\" "
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "nodeName is not ''",
    "condition_result": true
    }''')
task.configure(outputName="nodeStatus")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Force-Delete the Pod</h3>
<p>This action force deletes a pod. The pod may not be terminating due to a process that is not responding to a signal. The exact reason will be context-specific and application dependent. Common causes include:</p>
<ul>
<li>
<p>A tight loop in userspace code that does not allow for interrupt signals</p>
</li>
<li>
<p>A maintenance process (eg garbage collection) on the application runtime</p>
</li>
</ul>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>nodeStatus</code></p>
</blockquote>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl delete pod {terminatingPods.strip()} -n {namespace} --now\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "terminatingPods is not ''",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Get-List-of-Pods-in-Terminating-State"><a id="1" target="_self" rel="nofollow"></a>Check status of Node</h3>
<p>This action runs get pods command and if the specific pod no longer shows up when running&nbsp;<code>kubectl get pods, </code>then the issue has been <span style="color: rgb(45, 194, 107);">resolved</span>.</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>namespace</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>checkResolution</code></p>
</blockquote>
<p>&nbsp;</p>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -n {namespace}\\""
    }''')
task.configure(outputName="checkResolution")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

***
If the POD still stuck in `Terminating` state then you can consider.

    1. Restarting kubelet
        
        If you can SSH to the node and restart the kublet process. You may need
        administrator priveleges to do so. Before you do that, you may also want
        to check the kubelet logs for any issues.
        
    2. Check Whether finalizer's work needs to get done before termination
    
        This will vary depending on what the finalizer is doing. Please refer to 
        [Finalizers](https://kubernetes.io/docs/tasks/extend-kubernetes/custom-resources/custom-resource-definitions/#finalizers). Common cases for finalizers not completing realtes to
        Volumes.
        
***

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to identify pods stuck in Terminating State and removed the finalizer(if present) and tried force deletion of pod, using unSkript's K8s actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>